## ML_Job_skills

This Notebook is ML model can be used to predict the preferable category associated with the Qualification of the candidate.

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from keras.models import Model
from keras import regularizers
from keras.layers import Dense, Input, Dropout, LSTM, Activation,TimeDistributed,Reshape
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

## Data Extraction for predictions

Below cell is to prepare data set for the making ML model. For simpicity I have taken Preferred Qualification as a Input and Category as a model output.

In [31]:
df = pd.read_csv('Akshaykumar_Patel_machinelearning_job_skills.csv')
df_new = df['Category']
df1 = df_new.drop_duplicates()
df2 = df1.reset_index()
df2.drop('index',axis = 1, inplace  = True)
dict1 = df2.to_dict()
dict2 = dict1['Category']
dict2_inverted = dict(map(reversed, dict2.items()))
df3 = df[['Category','Preferred Qualifications']]
df3_list = df3.values.tolist()

X = []                             # This list will store the Preferred Qualifications 
for  j in range(len(df3_list)):
    X.append(df3_list[j][1])
X = np.array(X)
out_Y = []                            # This list will store Category associated with their Preferred Qualifications
for j in range(len(df3_list)):
    out_Y.append(dict2_inverted[df3_list[j][0]])
out_Y = np.array(out_Y) 
s= np.zeros((23))
Y=np.zeros((1250,23))
for j in range(len(out_Y)):
    Y[j][out_Y[j]] = 1 

## Trainig and Test data

Here there are 1250 different number of Qualifications so we can divide 1000 as training and rest 250 as validation set.
    

In [32]:
X_train=np.array(X[:999])
X_test=np.array(X[1000:])
Y_train=np.array(Y[:999])
Y_test=np.array(Y[1000:])

## Pre_processing for model generation

In [33]:
# I have used GlOVe vectors to vectorise the words 
def read_glove_vecs(glove_file):
    with open(glove_file,encoding="utf8") as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

word_to_index, index_to_word,word_to_vec_map = read_glove_vecs('Akshaykumar_Patel_machinelearning_glove.6B.50d.txt') # Reference : https://nlp.stanford.edu/projects/glove/

# finding max length of discription
max_len = len(X[0].strip().split())
for j in range(len(X)):
    if len(X[j].strip().split())>= max_len:
        max_len = len(X[j])
# max_len = 683. which means it contains 683 words. 
        
# This is to make make input ready for my model
def sentences_to_indices(X, word_to_index, max_len):
    m=X.shape[0]
    X_indices=np.zeros((m,max_len))
    for i in range(m):
        sentence_words= X[i].lower().split()
        j=0
        
        for w in sentence_words:
            if w not in word_to_index.keys():
                X_indices[i,j]=400001
            else:    
                X_indices[i,j]=word_to_index[w]
                j=j+1
    return X_indices    


## Model

## Model

In [34]:
def category_prediction(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(shape=input_shape, dtype='float32')
    X = Reshape((10,70))(sentence_indices)
    X = LSTM(128,return_sequences = True)(X)
    X = Dropout(0.2)(X)
    X = LSTM(128,kernel_regularizer=regularizers.l1(0.001),return_sequences = False)(X)
    X = Dropout(0.2)(X)
    X = Dense(23)(X)
    X = Activation('softmax')(X)
    model = Model(inputs=sentence_indices,outputs=X)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [35]:
model1 = category_prediction((700,), word_to_vec_map, word_to_index)
model1.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 700)               0         
_________________________________________________________________
reshape_6 (Reshape)          (None, 10, 70)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 10, 128)           101888    
_________________________________________________________________
dropout_11 (Dropout)         (None, 10, 128)           0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 23)                2967

In [36]:
X_train_indices = sentences_to_indices(X_train, word_to_index, 700)
model1.fit(X_train_indices, Y_train, epochs = 40, batch_size = 50, shuffle=True)

Epoch 1/40
999/999 [==============================] - 1s 1ms/step - loss: 5.4809 - accuracy: 0.1552
Epoch 2/40
999/999 [==============================] - 1s 535us/step - loss: 4.3125 - accuracy: 0.2032
Epoch 3/40
999/999 [==============================] - 1s 533us/step - loss: 3.4489 - accuracy: 0.2793
Epoch 4/40
999/999 [==============================] - 0s 485us/step - loss: 2.7796 - accuracy: 0.3754
Epoch 5/40
999/999 [==============================] - 0s 468us/step - loss: 2.3559 - accuracy: 0.4254
Epoch 6/40
999/999 [==============================] - 0s 471us/step - loss: 2.0675 - accuracy: 0.4995
Epoch 7/40
999/999 [==============================] - 0s 458us/step - loss: 1.8623 - accuracy: 0.5506
Epoch 8/40
999/999 [==============================] - 0s 456us/step - loss: 1.6689 - accuracy: 0.6056
Epoch 9/40
999/999 [==============================] - 0s 464us/step - loss: 1.4930 - accuracy: 0.6496
Epoch 10/40
999/999 [==============================] - 0s 454us/step - loss: 1.3640 

In [37]:
X_test_indices = sentences_to_indices(X_test, word_to_index, 700)
loss, acc = model1.evaluate(X_test_indices, Y_test)
print()
print("Test accuracy = ", acc)

250/250 [==============================] - 0s 575us/step

Test accuracy =  0.20399999618530273


As we can see the huge difference between training and test accuracy thus the model is overfitting.
We can do another approach for training and validation by using tratified K-fold ith 10 splits to validate the same model

In [38]:
# All classification reports will be added here. When we are done we can average the f1 scores
reports = []
y= np.zeros((1250))
# Apply stratified K-fold ith 10 splits. Stratified means the same distribution of classes than the whole dataset
# In this case, 50-50
kf = StratifiedKFold(n_splits=10)

# Just for printing purposes
id = 1

for train_index, test_index in kf.split(X,y):
    print('Kfold iteration {}/10'.format(id))
    id += 1 
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
        
    model2 = category_prediction((700,), word_to_vec_map, word_to_index)
    
    X_train_indices = sentences_to_indices(X_train, word_to_index, 700)
    X_test_indices = sentences_to_indices(X_test, word_to_index, 700)
    model2.fit(X_train_indices, y_train, epochs = 40, batch_size = 40, shuffle=True)
    loss,acc = model2.evaluate(X_test_indices, y_test) 
    reports.append(acc)

Kfold iteration 1/10
Epoch 1/40
1125/1125 [==============================] - 1s 1ms/step - loss: 5.3490 - accuracy: 0.1582
Epoch 2/40
1125/1125 [==============================] - 1s 566us/step - loss: 3.8066 - accuracy: 0.2613
Epoch 3/40
1125/1125 [==============================] - 1s 591us/step - loss: 2.9332 - accuracy: 0.3067
Epoch 4/40
1125/1125 [==============================] - 1s 587us/step - loss: 2.5369 - accuracy: 0.3680
Epoch 5/40
1125/1125 [==============================] - 1s 603us/step - loss: 2.3043 - accuracy: 0.43110s - loss: 2.2987 - accu
Epoch 6/40
1125/1125 [==============================] - 1s 598us/step - loss: 2.0964 - accuracy: 0.4738
Epoch 7/40
1125/1125 [==============================] - 1s 608us/step - loss: 1.9173 - accuracy: 0.5084
Epoch 8/40
1125/1125 [==============================] - 1s 611us/step - loss: 1.7503 - accuracy: 0.5742
Epoch 9/40
1125/1125 [==============================] - 1s 616us/step - loss: 1.6068 - accuracy: 0.6187
Epoch 10/40
1125/1125

1125/1125 [==============================] - 1s 579us/step - loss: 0.2050 - accuracy: 0.9893
Epoch 38/40
1125/1125 [==============================] - 1s 580us/step - loss: 0.1944 - accuracy: 0.9893
Epoch 39/40
1125/1125 [==============================] - 1s 570us/step - loss: 0.1989 - accuracy: 0.9876
Epoch 40/40
125/125 [==============================] - 0s 990us/step
Kfold iteration 3/10
Epoch 1/40
1125/1125 [==============================] - 1s 1ms/step - loss: 5.3374 - accuracy: 0.1476
Epoch 2/40
1125/1125 [==============================] - 1s 603us/step - loss: 3.8675 - accuracy: 0.2498
Epoch 3/40
1125/1125 [==============================] - 1s 586us/step - loss: 3.0536 - accuracy: 0.2676
Epoch 4/40
1125/1125 [==============================] - 1s 601us/step - loss: 2.5890 - accuracy: 0.3529
Epoch 5/40
1125/1125 [==============================] - 1s 592us/step - loss: 2.3835 - accuracy: 0.3813
Epoch 6/40
1125/1125 [==============================] - 1s 590us/step - loss: 2.2082 - ac

1125/1125 [==============================] - 1s 558us/step - loss: 0.2794 - accuracy: 0.9804
Epoch 34/40
1125/1125 [==============================] - 1s 558us/step - loss: 0.2506 - accuracy: 0.9867
Epoch 35/40
1125/1125 [==============================] - 1s 560us/step - loss: 0.2497 - accuracy: 0.9822
Epoch 36/40
1125/1125 [==============================] - 1s 552us/step - loss: 0.2436 - accuracy: 0.9840
Epoch 37/40
1125/1125 [==============================] - 1s 545us/step - loss: 0.2639 - accuracy: 0.9804
Epoch 38/40
1125/1125 [==============================] - 1s 553us/step - loss: 0.2731 - accuracy: 0.9760
Epoch 39/40
1125/1125 [==============================] - 1s 560us/step - loss: 0.2253 - accuracy: 0.9867
Epoch 40/40
125/125 [==============================] - 0s 916us/step
Kfold iteration 5/10
Epoch 1/40
1125/1125 [==============================] - 1s 1ms/step - loss: 5.4093 - accuracy: 0.1093
Epoch 2/40
1125/1125 [==============================] - 1s 644us/step - loss: 3.9546 

1125/1125 [==============================] - 1s 638us/step - loss: 0.3143 - accuracy: 0.9751
Epoch 30/40
1125/1125 [==============================] - 1s 645us/step - loss: 0.2858 - accuracy: 0.9778
Epoch 31/40
1125/1125 [==============================] - 1s 649us/step - loss: 0.2528 - accuracy: 0.9867
Epoch 32/40
1125/1125 [==============================] - 1s 637us/step - loss: 0.2355 - accuracy: 0.9858
Epoch 33/40
1125/1125 [==============================] - 1s 640us/step - loss: 0.2351 - accuracy: 0.9849
Epoch 34/40
1125/1125 [==============================] - 1s 643us/step - loss: 0.2648 - accuracy: 0.9787
Epoch 35/40
1125/1125 [==============================] - 1s 646us/step - loss: 0.2849 - accuracy: 0.9796
Epoch 36/40
1125/1125 [==============================] - 1s 649us/step - loss: 0.2680 - accuracy: 0.9733
Epoch 37/40
1125/1125 [==============================] - 1s 617us/step - loss: 0.2488 - accuracy: 0.9840
Epoch 38/40
1125/1125 [==============================] - 1s 650us/s

1125/1125 [==============================] - 1s 684us/step - loss: 0.3204 - accuracy: 0.9760
Epoch 26/40
1125/1125 [==============================] - 1s 678us/step - loss: 0.2922 - accuracy: 0.9804
Epoch 27/40
1125/1125 [==============================] - 1s 671us/step - loss: 0.3018 - accuracy: 0.9769
Epoch 28/40
1125/1125 [==============================] - 1s 671us/step - loss: 0.3978 - accuracy: 0.9502
Epoch 29/40
1125/1125 [==============================] - 1s 679us/step - loss: 0.3767 - accuracy: 0.9511
Epoch 30/40
1125/1125 [==============================] - 1s 673us/step - loss: 0.3652 - accuracy: 0.96090s - loss: 0.3648 - accuracy
Epoch 31/40
1125/1125 [==============================] - 1s 675us/step - loss: 0.4311 - accuracy: 0.9413
Epoch 32/40
1125/1125 [==============================] - 1s 673us/step - loss: 0.4101 - accuracy: 0.9529
Epoch 33/40
1125/1125 [==============================] - 1s 669us/step - loss: 0.3298 - accuracy: 0.9680
Epoch 34/40
1125/1125 [================

1125/1125 [==============================] - 1s 742us/step - loss: 0.5505 - accuracy: 0.9111
Epoch 22/40
1125/1125 [==============================] - 1s 737us/step - loss: 0.5554 - accuracy: 0.9138
Epoch 23/40
1125/1125 [==============================] - 1s 744us/step - loss: 0.4855 - accuracy: 0.9342
Epoch 24/40
1125/1125 [==============================] - 1s 746us/step - loss: 0.4430 - accuracy: 0.94580s - loss: 0.4473 - accu
Epoch 25/40
1125/1125 [==============================] - 1s 740us/step - loss: 0.3971 - accuracy: 0.9609
Epoch 26/40
1125/1125 [==============================] - 1s 750us/step - loss: 0.3507 - accuracy: 0.9698
Epoch 27/40
1125/1125 [==============================] - 1s 741us/step - loss: 0.3164 - accuracy: 0.9769
Epoch 28/40
1125/1125 [==============================] - 1s 741us/step - loss: 0.2976 - accuracy: 0.9778
Epoch 29/40
1125/1125 [==============================] - 1s 745us/step - loss: 0.2821 - accuracy: 0.9760
Epoch 30/40
1125/1125 [====================

In [39]:
print('Mean Accuracy is:',100*np.mean(reports),'%')

Mean Accuracy is: 41.04000061750412 %
